# Intent Classification  &  Entity Name Recognition

## Intent Classification
Is a part of Natural Language Understanding, where the machine algorithm learns to classify a given phrase on the basis of the ones it has been trained on

## Entity Name Recognition
Is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

In [1]:
import numpy as np
import pickle
import random
import json
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intent']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
print("*"*50)

# classes = intents
print (len(classes), "classes", classes)
print("*"*50)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

75 documents
**************************************************
12 classes ['alarm', 'date', 'getweather', 'goodbye', 'greeting', 'info', 'options', 'play_music', 'thanks', 'time', 'turnOFFlight', 'turnONlight']
**************************************************
147 unique lemmatized words ["'s", ',', '11', '4', '5', '5:45', '6:15', '6:30', '7', '8:00', '9', '[', ']', 'a', 'about', 'alarm', 'am', 'an', 'and', 'another', 'anyone', 'are', 'ask', 'at', 'awesome', 'be', 'brief', 'bye', 'cairo', 'can', 'clock', 'cookie', 'could', 'create', 'current', 'date', 'day', 'definition', 'dim', 'dinner', 'do', 'doe', 'doing', 'for', 'forecast', 'from', 'gaga', 'garage', 'get', 'give', 'go', 'going', 'good', 'goodbye', 'hear', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hour', 'how', 'i', 'in', 'information', 'is', 'it', 'jazz', 'kitchen', 'know', 'lady', 'later', 'let', 'light', 'like', 'listen', 'looking', 'lunch', 'main', 'make', 'maroon', 'me', 'minute', 'music', 'my', 'need', 'n

In [3]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-3-0083ae827ca8>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [4]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [5]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\Mohamed Nabil\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [6]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
15/15 [==============================] - 0s 854us/step - loss: 2.5274 - accuracy: 0.0933
Epoch 2/200
15/15 [==============================] - 0s 857us/step - loss: 2.3960 - accuracy: 0.1067
Epoch 3/200
15/15 [==============================] - 0s 859us/step - loss: 2.3394 - accuracy: 0.2000
Epoch 4/200
15/15 [==============================] - 0s 744us/step - loss: 2.2028 - accuracy: 0.2400
Epoch 5/200
15/15 [==============================] - 0s 1ms/step - loss: 2.0089 - accuracy: 0.4000
Epoch 6/200
15/15 [==============================] - 0s 1ms/step - loss: 1.8494 - accuracy: 0.3200
Epoch 7/200
15/15 [==============================] - 0s 1ms/step - loss: 1.8080 - accuracy: 0.3733
Epoch 8/200
15/15 [==============================] - 0s 574us/step - loss: 1.5199 - accuracy: 0.5067
Epoch 9/200
15/15 [==============================] - 0s 858us/step - loss: 1.4888 - accuracy: 0.5467
Epoch 10/200
15/15 [==============================] - 0s 1ms/step - loss: 1.4445 - accuracy: 0.56

15/15 [==============================] - 0s 712us/step - loss: 0.0568 - accuracy: 0.9733
Epoch 83/200
15/15 [==============================] - 0s 712us/step - loss: 0.0488 - accuracy: 0.9867
Epoch 84/200
15/15 [==============================] - 0s 570us/step - loss: 0.0524 - accuracy: 1.0000
Epoch 85/200
15/15 [==============================] - 0s 712us/step - loss: 0.0763 - accuracy: 0.9600
Epoch 86/200
15/15 [==============================] - 0s 641us/step - loss: 0.0807 - accuracy: 0.9733
Epoch 87/200
15/15 [==============================] - 0s 712us/step - loss: 0.0343 - accuracy: 0.9867
Epoch 88/200
15/15 [==============================] - 0s 570us/step - loss: 0.0525 - accuracy: 1.0000
Epoch 89/200
15/15 [==============================] - 0s 712us/step - loss: 0.0726 - accuracy: 0.9733
Epoch 90/200
15/15 [==============================] - 0s 641us/step - loss: 0.0242 - accuracy: 1.0000
Epoch 91/200
15/15 [==============================] - 0s 641us/step - loss: 0.0529 - accuracy: 

15/15 [==============================] - 0s 857us/step - loss: 0.0710 - accuracy: 0.9600
Epoch 162/200
15/15 [==============================] - 0s 857us/step - loss: 0.0609 - accuracy: 0.9867
Epoch 163/200
15/15 [==============================] - 0s 857us/step - loss: 0.0256 - accuracy: 1.0000
Epoch 164/200
15/15 [==============================] - 0s 571us/step - loss: 0.0384 - accuracy: 0.9867
Epoch 165/200
15/15 [==============================] - 0s 857us/step - loss: 0.0251 - accuracy: 1.0000
Epoch 166/200
15/15 [==============================] - 0s 857us/step - loss: 0.0355 - accuracy: 1.0000
Epoch 167/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0243 - accuracy: 1.0000
Epoch 168/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 169/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0295 - accuracy: 1.0000
Epoch 170/200
15/15 [==============================] - 0s 857us/step - loss: 0.0193 - accurac

In [7]:
model.save('Model.h5',hist)

with open('classes.pkl','wb') as file:  
    pickle.dump(classes,file)

with open('words.pkl','wb') as file:  
    pickle.dump(words,file)


In [8]:
from keras.models import load_model
model = load_model('Model.h5')

with open('classes.pkl','rb') as file:  
    classes = pickle.load(file)

with open('words.pkl','rb') as file:  
    words = pickle.load(file)


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"Intent": classes[r[0]], "Probability": str(r[1])})
    return return_list

In [9]:
predict_class('what is the weather in cairo?',model)

[{'Intent': 'getweather', 'Probability': '0.9999659'}]

In [10]:
predict_class('please turn on the light of room',model)

[{'Intent': 'turnONlight', 'Probability': '0.9999627'}]

In [11]:
def ner(sentence):
    result = []
    for ent in nlp(sentence).ents:
        result.append({"Name": ent.text , "Label": ent.label_})
    return result

In [12]:
def nlu(sentence):
    intent = predict_class(sentence,model)
    entity = ner(sentence)
    print(intent,entity)

In [13]:
nlu('What is the time now in cairo?')

[{'Intent': 'time', 'Probability': '0.99999976'}] [{'Name': 'cairo', 'Label': 'GPE'}]


In [14]:
nlu('Do you feel different because of weather?')

[{'Intent': 'getweather', 'Probability': '0.95905155'}] []


In [15]:
nlu("Do you know what the weather will be like tomorrow?")

[{'Intent': 'getweather', 'Probability': '0.99956137'}] [{'Name': 'tomorrow', 'Label': 'DATE'}]


In [16]:
nlu( "Do you know if it's going to rain tomorrow?" )

[{'Intent': 'getweather', 'Probability': '0.9300208'}] [{'Name': 'tomorrow', 'Label': 'DATE'}]


In [17]:
nlu('Tell me some information about History of Ancient Egyptian')

[{'Intent': 'info', 'Probability': '0.9191558'}] []


In [18]:
nlu('Get an information about climate changes')

[{'Intent': 'greeting', 'Probability': '0.6700375'}, {'Intent': 'info', 'Probability': '0.3023287'}] []
